<a href="https://colab.research.google.com/github/Kadomium/MSTB/blob/main/find_best_bombination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [27]:
df = pd.read_csv('date_raw.csv')
df.head()
#頭2行を削除
df = df.drop([0, 1], axis=0).reset_index(drop=True)
#14行目以降を削除
df = df.iloc[:, :13]
#名前の付け直し
df = df.set_axis(['date', 'yen_usd_open', 'yen_usd_high', 'yen_usd_low', 'yen_usd_last','eur_usd_open', 'eur_usd_high', 'eur_usd_low', 'eur_usd_last','eur_yen_open', 'eur_yen_high', 'eur_yen_low', 'eur_yen_last'],axis='columns')
#数字を文字列ではなく数字と認識してもらう
df[['yen_usd_open', 'yen_usd_high', 'yen_usd_low', 'yen_usd_last','eur_usd_open', 'eur_usd_high', 'eur_usd_low', 'eur_usd_last','eur_yen_open', 'eur_yen_high', 'eur_yen_low', 'eur_yen_last']] = df[['yen_usd_open', 'yen_usd_high', 'yen_usd_low', 'yen_usd_last','eur_usd_open', 'eur_usd_high', 'eur_usd_low', 'eur_usd_last','eur_yen_open', 'eur_yen_high', 'eur_yen_low', 'eur_yen_last']].apply(pd.to_numeric, errors='coerce')
df.head()

,date,yen_usd_open,yen_usd_high,yen_usd_low,yen_usd_last,eur_usd_open,eur_usd_high,eur_usd_low,eur_usd_last,eur_yen_open,eur_yen_high,eur_yen_low,eur_yen_last
0,2000/1/1,102.45,102.59,102.12,102.51,1.0090,1.0136,1.0046,1.0062,103.49,103.51,102.24,103.08
1,2000/1/2,102.45,102.59,102.12,102.51,1.0090,1.0136,1.0046,1.0062,103.49,103.51,102.24,103.08
2,2000/1/3,102.14,102.17,101.40,101.45,1.0073,1.0285,1.0052,1.0243,102.89,104.30,102.17,103.93
3,2000/1/4,101.70,103.30,101.46,103.22,1.0244,1.0341,1.0217,1.0296,104.22,106.56,104.01,106.34
4,2000/1/5,103.17,104.40,102.81,104.14,1.0300,1.0403,1.0285,1.0321,105.77,107.74,105.63,107.63


In [28]:
columns_to_drop = [col for col in df.columns if 'open' in col or 'high' in col or 'low' in col]
df = df.drop(columns=columns_to_drop)

In [29]:
def calculate_rolling_average(column_name_short, column_name_long, df):
    # 列名からnを取得
    n = int(column_name_short.split('_')[-1][2:])  # 'yen_usd_MA5' なら 5 を取得
    m = int(column_name_long.split('_')[-1][2:])  # 'yen_usd_MA50' なら 50 を取得

    # 過去nつの値の平均を計算
    df[column_name_short] = df['yen_usd_last'].rolling(window=n).mean()
    df[column_name_long] = df['yen_usd_last'].rolling(window=n).mean()

    # 最初のn-1行をn行目の値で埋める
    if len(df) >= n:
        df.loc[:n-2, column_name] = df.loc[n-1, column_name]

    return df

In [30]:
def get_line(column_name_short, column_name_long, df):


  n = int(column_name_short.split('_')[-1][2:])  # 'yen_usd_MA5' なら 5 を取得
  m = int(column_name_long.split('_')[-1][2:])  # 'yen_usd_MA50' なら 50 を取得
   # 過去nつの値の平均を計算

 # 過去nつの値の平均を計算
  df[column_name_short] = df['yen_usd_last'].rolling(window=n).mean()
  if len(df) >= n:
        df.loc[:n-2, column_name_short] = df.loc[n-1, column_name_short]

  df[column_name_long] = df['yen_usd_last'].rolling(window=n).mean()
  if len(df) >= n:
        df.loc[:n-2, column_name_long] = df.loc[n-1, column_name_long]
  df[f'prev_yen_usd_{n}'] = df[column_name_short].shift(1)
  df[f'prev_yen_usd_{m}'] = df[column_name_long].shift(1)
  df[f'yen_usd_sell_{n}_{m}'] = df.apply(
      lambda row: row['yen_usd_last'] if row[f'column_name_{n}'] >= row[f'yen_usd_{m}'] and row[f'prev_yen_usd_{m}'] >= row[f'prev_yen_usd_{n}'] else 0,
      axis=1)
  df[f'yen_usd_buy_{n}_{m}'] = df.apply(
      lambda row: row['yen_usd_last'] if row[f'yen_usd_{m}'] >= row[f'yen_usd_{n}'] and row[f'prev_yen_usd_{n}'] >= row[f'prev_yen_usd_{m}'] else 0,
      axis=1
      )
  df[f'yen_usd_sell_num_{n}_{m}'] = (df[f'yen_usd_sell_{n}_{m}'] != 0).cumsum()
  df[f'yen_usd_buy_num_{n}_{m}'] = (df[f'yen_usd_buy_{n}_{m}'] != 0).cumsum()
  df[f'yen_usd_buy_over_{n}_{m}'] = df[f'yen_usd_buy_num_{n}_{m}'] - df[f'yen_usd_sell_num_{n}_{m}']
  df[f'yen_usd_profit_{n}_{m}'] = 0
  #全部1を売ってトレードする
  for i in range(1, len(df)):
    df.loc[i, f'yen_usd_profit_{n}_{m}'] = (
        df.loc[i - 1, f'yen_usd_profit_{n}_{m}'] - df.loc[i, f'yen_usd_buy_{n}_{m}'] + df.loc[i, f'yen_usd_sell_{n}_{m}']
    )
  #買いが超過していた場合、最後に売る
  if df.iloc[-1][f'yen_usd_buy_over_{n}_{m}'] > 0:
    df.loc[df.index[-1], f'yen_usd_profit_{n}_{m}'] += df.iloc[-1]['yen_usd_last']
  return df.iloc[-1][f'yen_usd_profit_{n}_{m}']

In [33]:
def get_line(column_name_short, column_name_long, df):
    n = int(column_name_short.split('_')[-1][2:])  # 'yen_usd_MA5' なら 5 を取得
    m = int(column_name_long.split('_')[-1][2:])  # 'yen_usd_MA50' なら 50 を取得

    # 過去nつの値の平均を計算
    df[column_name_short] = df['yen_usd_last'].rolling(window=n).mean()
    if len(df) >= n:
        df.loc[:n-2, column_name_short] = df.loc[n-1, column_name_short]

    df[column_name_long] = df['yen_usd_last'].rolling(window=m).mean()
    if len(df) >= m:
        df.loc[:m-2, column_name_long] = df.loc[m-1, column_name_long]

    df[f'prev_yen_usd_{n}'] = df[column_name_short].shift(1)
    df[f'prev_yen_usd_{m}'] = df[column_name_long].shift(1)

    # 買い・売り条件を満たす場合の列を作成
    df[f'yen_usd_sell_{n}_{m}'] = df.apply(
        lambda row: row['yen_usd_last'] if row[column_name_short] >= row[column_name_long] and row[f'prev_yen_usd_{m}'] >= row[f'prev_yen_usd_{n}'] else 0,
        axis=1
    )
    df[f'yen_usd_buy_{n}_{m}'] = df.apply(
        lambda row: row['yen_usd_last'] if row[column_name_long] >= row[column_name_short] and row[f'prev_yen_usd_{n}'] >= row[f'prev_yen_usd_{m}'] else 0,
        axis=1
    )

    # 買い・売りの回数をカウント
    df[f'yen_usd_sell_num_{n}_{m}'] = (df[f'yen_usd_sell_{n}_{m}'] != 0).cumsum()
    df[f'yen_usd_buy_num_{n}_{m}'] = (df[f'yen_usd_buy_{n}_{m}'] != 0).cumsum()
    df[f'yen_usd_buy_over_{n}_{m}'] = df[f'yen_usd_buy_num_{n}_{m}'] - df[f'yen_usd_sell_num_{n}_{m}']

    # 損益計算
    df[f'yen_usd_profit_{n}_{m}'] = 0
    for i in range(1, len(df)):
        df.loc[i, f'yen_usd_profit_{n}_{m}'] = (
            df.loc[i - 1, f'yen_usd_profit_{n}_{m}'] - df.loc[i, f'yen_usd_buy_{n}_{m}'] + df.loc[i, f'yen_usd_sell_{n}_{m}']
        )

    # 最終行で買いが超過していた場合、最後に売る
    if df.iloc[-1][f'yen_usd_buy_over_{n}_{m}'] > 0:
        df.loc[df.index[-1], f'yen_usd_profit_{n}_{m}'] += df.iloc[-1]['yen_usd_last']

    # 最終行の利益値を返す
    return df.iloc[-1][f'yen_usd_profit_{n}_{m}']


In [39]:
def get_result(column_name_short, column_name_long, df):
    # パラメータを解析
    n = int(column_name_short.split('_')[-1][2:])  # 'yen_usd_MA5' なら 5 を取得
    m = int(column_name_long.split('_')[-1][2:])  # 'yen_usd_MA50' なら 50 を取得

    # 過去nつ/mつの平均を計算
    ma_short = df['yen_usd_last'].rolling(window=n).mean()
    ma_long = df['yen_usd_last'].rolling(window=m).mean()

    # 1行前のMA値を計算
    prev_ma_short = ma_short.shift(1)
    prev_ma_long = ma_long.shift(1)

    # 売り・買い条件を計算
    sell = (ma_short >= ma_long) & (prev_ma_long >= prev_ma_short)
    buy = (ma_long >= ma_short) & (prev_ma_short >= prev_ma_long)

    # 売り・買い値を記録
    yen_usd_sell = df['yen_usd_last'][sell].reindex(df.index, fill_value=0)
    yen_usd_buy = df['yen_usd_last'][buy].reindex(df.index, fill_value=0)

    # 利益を計算
    profit = 0
    for i in range(1, len(df)):
        profit = profit - yen_usd_buy.iloc[i] + yen_usd_sell.iloc[i]

    # 最終行で買い越し分を追加
    buy_over = buy.cumsum() - sell.cumsum()
    if buy_over.iloc[-1] > 0:
        profit += df['yen_usd_last'].iloc[-1]

    # 最終行の利益を返す
    return profit


In [42]:
get_result('yen_usd_MA25', 'yen_usd_MA200', df)

123.68000000000002

In [43]:
# 最適な組み合わせを探す関数
def find_best_combination(df):
    best_profit = float('-inf')  # 初期値を最小値に設定
    best_n, best_m = None, None  # 最適な n, m を保存

    for n in range(1, 501):  # n を 1 から 500 まで
        for m in range(1, 501):  # m を 1 から 500 まで
            # m > n の場合のみ計算（条件を付けて効率化）
            if m > n:
                try:
                    # get_line を実行して利益を取得
                    profit = get_result(f'yen_usd_MA{n}', f'yen_usd_MA{m}', df)

                    # 利益が最も高い場合、更新
                    if profit > best_profit:
                        best_profit = profit
                        best_n, best_m = n, m
                except Exception as e:
                    print(f"n={n}, m={m} でエラーが発生: {e}")

    return best_profit, best_n, best_m


In [ ]:
find_best_combination(df)